In [2]:
pwd

'/home/wsuser/work'

In [3]:
!pip install keras
!pip install tensorflow

In [4]:
#import Keras Library
import keras
#import ImageDataGenerator class from keras
from keras.preprocessing.image import ImageDataGenerator

In [5]:
#Define The Parameters /Arguments For ImageDataGenerator Class
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,rotation_range=180,zoom_range=0.2,horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)

In [6]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='NMvJHmV_e_JPe-wztWEj0YDnqkVghDQuZjRpVi31tsDc',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'trainimageclassificationmodel-donotdelete-pr-e69sqwhlmq1zzl'
object_key = 'archive.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [7]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [8]:
ls

Dataset/  forest1.h5


In [9]:
pwd

'/home/wsuser/work'

In [10]:
#Applying ImageDataGenerator functionality to trainset.
x_train = train_datagen.flow_from_directory(r"/home/wsuser/work/Dataset/Dataset/train_set",target_size = (128,128),batch_size = 32,class_mode = 'binary')
#Applying ImageDataGenerator functionality to testset.
x_test = test_datagen.flow_from_directory(r"/home/wsuser/work/Dataset/Dataset/test_set",target_size = (128,128),batch_size = 32,class_mode = 'binary')

Found 436 images belonging to 2 classes.
Found 121 images belonging to 2 classes.


In [11]:
#import model building libraries...
#To define linear intialisation import sequential
from keras.models import Sequential
#To add layers import Dense
from keras.layers import Dense
#To create convolution kernal import convolution2D
from keras.layers import Convolution2D
#import Maxpooling layer
from keras.layers import MaxPooling2D
#import flatten layer
from keras.layers import Flatten
import warnings
warnings.filterwarnings('ignore')

In [12]:
#initializing the model
model = Sequential()

In [13]:
#add convolutional layer
model.add (Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))

In [14]:
#add maxpooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

In [15]:
#add flatten layer
model.add(Flatten())

In [16]:
#add hidden layer
model.add(Dense(150,activation='relu'))
#add output layer
model.add(Dense(1,activation='sigmoid'))

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 127008)            0         
                                                                 
 dense (Dense)               (None, 150)               19051350  
                                                                 
 dense_1 (Dense)             (None, 1)                 151       
                                                                 
Total params: 19,052,397
Trainable params: 19,052,397
Non-trainable params: 0
____________________________________________

In [18]:
#configure the learning process
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ["accuracy"])

In [19]:
#training the model
model.fit_generator(x_train,steps_per_epoch=14,epochs=10,validation_data=x_test,validation_steps=4)

Epoch 1/10
14/14 [==============================] - 28s 2s/step - loss: 5.6319 - accuracy: 0.6124 - val_loss: 0.6531 - val_accuracy: 0.8017
Epoch 2/10
14/14 [==============================] - 27s 2s/step - loss: 0.8608 - accuracy: 0.7385 - val_loss: 0.2385 - val_accuracy: 0.8843
Epoch 3/10
14/14 [==============================] - 27s 2s/step - loss: 0.2617 - accuracy: 0.8807 - val_loss: 0.1466 - val_accuracy: 0.9504
Epoch 4/10
14/14 [==============================] - 26s 2s/step - loss: 0.2065 - accuracy: 0.9060 - val_loss: 0.0969 - val_accuracy: 0.9752
Epoch 5/10
14/14 [==============================] - 26s 2s/step - loss: 0.1693 - accuracy: 0.9289 - val_loss: 0.1040 - val_accuracy: 0.9669
Epoch 6/10
14/14 [==============================] - 26s 2s/step - loss: 0.1704 - accuracy: 0.9220 - val_loss: 0.0969 - val_accuracy: 0.9752
Epoch 7/10
14/14 [==============================] - 26s 2s/step - loss: 0.1775 - accuracy: 0.9358 - val_loss: 0.0788 - val_accuracy: 0.9835
Epoch 8/10
14/14 [==

In [23]:
#save the model
model.save("forest.h5")

In [20]:
!pip install watson-machine-learning-client 

     |████████████████████████████████| 538 kB 21.6 MB/s eta 0:00:01


In [22]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"iYHz8VT75CDTbRUnBRLhZZ7K5blvuaTmiPcPZZxkndLH"
}

client=APIClient(wml_credentials)

In [24]:
client

In [28]:
def guid_space_name(client,animal_deploy):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==animal_deploy)['metadata']['id'])

In [29]:
space_uid=guid_space_name(client,'Train Image Classification Model')
print("Space UID "+space_uid)

Space UID 43b82e7a-6aba-4cea-8d2a-0859affa230b


In [30]:
client.set.default_space(space_uid)

'SUCCESS'

In [31]:
client.software_specifications.list(200)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

In [32]:
software_space_uid=client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')

In [33]:
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [34]:
ls

Dataset/  forest1.h5


In [41]:
!tar -zcvf image-Classification-model.tgz forest1.h5

forest1.h5


In [42]:
model_details=client.repository.store_model(model='image-Classification-model.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN Model Building",
    client.repository.ModelMetaNames.TYPE:'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})

In [43]:
model_id=client.repository.get_model_id(model_details)

In [44]:
model_id

'83d1e9b9-9fa3-430a-9a31-6dc5768b1081'

In [45]:
client.repository.download(model_id,'forest1.tar.gb')

Successfully saved model content to file: 'forest1.tar.gb'


'/home/wsuser/work/forest1.tar.gb'